# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print("currrent work directory:", os.getcwd())
filepath = os.getcwd() + '/event_data'
file_path_list = glob.glob(os.path.join(filepath,'*.csv')) 

currrent work directory: /home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
# extract and union data from different csv files
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader) #read header
        full_data_rows_list.extend(csvreader)
        
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
sparkify_keyspace_create = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : %s, 'replication_factor' : %s }
                """
try:
    session.execute(sparkify_keyspace_create, ("SimpleStrategy", 1))
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## QUERY 1

`
Query Description: In this query, I used sessionId as the partition key and itemInSession as my clustering key. Each partition is uniquely identified by sessionId while itemInSession was used to uniquely identify the rows within a partition to sort the data by the value of number items in session .
`
  
  
`
Example: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
`

#### preparing data step

In [8]:
#create table
session_item_table_drop = "DROP TABLE  IF EXISTS session_item"
session_item_table_create = """CREATE TABLE IF NOT EXISTS session_item(
                               sessionId int, 
                               itemInSession int,
                               artist text, 
                               song text, 
                               length float, 
                               PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(session_item_table_drop)
    session.execute(session_item_table_create)
except Exception as e:
    print(e)

# load data
session_item_insert = "INSERT INTO session_item(sessionId, itemInSession, artist, song, length) VALUES (%s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)
    name_to_position={header[position]: position for position in range(len(header))}
    for line in csvreader:
        session_id = int(line[name_to_position['sessionId']])
        item_in_session = int(line[name_to_position['itemInSession']])
        artist = line[name_to_position['artist']]
        song = line[name_to_position['song']]
        length = float(line[name_to_position['length']])
        values = (session_id, item_in_session, artist, song, length)
        session.execute(session_item_insert, values)

#### query step

In [9]:
# query on table session_item
query = """SELECT artist, song, length FROM session_item WHERE sessionId = 338 AND itemInSession = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print( row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### check insertion step

In [10]:
session_item_count = "SELECT COUNT(*) FROM session_item"
try:
    result = session.execute(session_item_count)
    print("session_item number of rows:", result[0].count)
except Exception as e:
    print(e)


session_item number of rows: 6820


## QUERY 2


`
Query Description: In this query, I used (sessionId, userId) as the partition key and itemInSession as my clustering key. Each partition is uniquely identified by (sessionId, userId) while itemInSession was used to get them in sorted order.
`
  
`Example: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182`


#### preparing data step

In [11]:
#create table
user_session_table_drop = "DROP TABLE IF EXISTS user_session"
user_session_table_create = """CREATE TABLE IF NOT EXISTS user_session(
                               sessionId int, 
                               userId int,
                               artist text, 
                               song text, 
                               firstName text,
                               lastName text,
                               itemInSession int, 
                               PRIMARY KEY ((sessionId, userId), itemInSession))"""
try:
    session.execute(user_session_table_drop)
    session.execute(user_session_table_create)
except Exception as e:
    print(e)


# load data
user_session_insert = "INSERT INTO user_session(sessionId, userId, artist, song, firstName, lastName, itemInSession) VALUES (%s, %s, %s, %s, %s, %s, %s)"

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)
    name_to_position={header[position]: position for position in range(len(header))}
    for line in csvreader:
        session_id = int(line[name_to_position['sessionId']])
        user_id = int(line[name_to_position['userId']])
        artist = line[name_to_position['artist']]
        song = line[name_to_position['song']]
        first_name = line[name_to_position['firstName']]
        last_name = line[name_to_position['lastName']]                  
        item_in_session = int(line[name_to_position['itemInSession']])
        values = (session_id, user_id, artist, song, first_name, last_name, item_in_session)
        session.execute(user_session_insert, values)

#### query step

In [12]:
query = """SELECT artist, song, firstName, lastName FROM user_session WHERE userId = 10 AND sessionId = 182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname )

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### check insertion step

In [13]:
user_session_count = "SELECT COUNT(*) FROM user_session"
try:
    result = session.execute(user_session_count)
    print("session_item number of rows:", result[0].count)
except Exception as e:
    print(e)


session_item number of rows: 6820


## QUERY 3
`
Query Description: In this query, I used song as the partition key and userId as my clustering key. Each partition is uniquely identified by song while userId was used to uniquely identify the rows within a partition.
`

`Example: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own`

#### preparing data step

In [14]:
song_user_table_drop = "DROP TABLE IF EXISTS song_user"
song_user_table_create = """CREATE TABLE IF NOT EXISTS song_user(
                               song text, 
                               userId int,                               
                               firstName text,
                               lastName text,
                               PRIMARY KEY (song, userId))"""
try:
    session.execute(song_user_table_drop)
    session.execute(song_user_table_create)
except Exception as e:
    print(e)


# load data
song_user_insert = "INSERT INTO song_user(song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    header = next(csvreader)
    name_to_position={header[position]: position for position in range(len(header))}
    for line in csvreader:
        song = line[name_to_position['song']]
        user_id = int(line[name_to_position['userId']])
        first_name = line[name_to_position['firstName']]
        last_name = line[name_to_position['lastName']]                  
        values = (song, user_id, first_name, last_name)
        session.execute(song_user_insert, values)
                    

#### query step

In [15]:
query = """SELECT firstName, lastName FROM song_user WHERE song = 'All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print( row.firstname, row.lastname )

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### check insertion step

In [16]:
user_session_count = "SELECT COUNT(*) FROM song_user"
try:
    result = session.execute(user_session_count)
    print("session_item number of rows:", result[0].count)
except Exception as e:
    print(e)

session_item number of rows: 6618


### Drop the tables before closing out the sessions

In [17]:

try:
    session.execute(session_item_table_drop)
    session.execute(user_session_table_drop)
    session.execute(song_user_table_drop)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()